## Patrick Adams 
## 1231065
### DTSC 620 Project 2


## Part 1: Fuse Decision Tree, Gaussian Naive Bayes, and Logistic Regression with majority ruling

In [28]:
#imports
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [5]:
# loading the dataset into a dataframe
spam_fused = pd.read_csv('spam.csv')

In [6]:
spam_fused.head()

,make,address,all,3d,our,over,remove,internet,order,mail,...,semicol,paren,bracket,bang,dollar,pound,cap_avg,cap_long,cap_total,Class
0,0.00,0.00,0.29,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.178,0.0,0.044,0.000,0.00,1.666,10,180,ham
1,0.46,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.125,0.0,0.000,0.000,0.00,1.510,10,74,ham
2,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.00,1.718,11,55,ham
3,0.33,0.44,0.37,0.0,0.14,0.11,0.00,0.07,0.97,1.16,...,0.006,0.159,0.0,0.069,0.221,0.11,3.426,72,819,spam
4,0.00,2.08,0.00,0.0,3.12,0.00,1.04,0.00,0.00,0.00,...,0.000,0.000,0.0,0.263,0.000,0.00,1.428,4,20,spam


In [10]:
# #Using LabelEncoder to transform categorical data
class_le = spam_fused[["Class"]].apply(LabelEncoder().fit_transform)

In [12]:
#dropping original categorical data
spam_fused.drop(["Class"], axis = 1, inplace=True)

In [14]:
#merge dummy variables with the main dataframe
spam_fused = pd.concat([spam_fused, class_le], axis=1)

In [15]:
spam_fused.head()

,make,address,all,3d,our,over,remove,internet,order,mail,...,semicol,paren,bracket,bang,dollar,pound,cap_avg,cap_long,cap_total,Class
0,0.00,0.00,0.29,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.178,0.0,0.044,0.000,0.00,1.666,10,180,0
1,0.46,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.125,0.0,0.000,0.000,0.00,1.510,10,74,0
2,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.00,1.718,11,55,0
3,0.33,0.44,0.37,0.0,0.14,0.11,0.00,0.07,0.97,1.16,...,0.006,0.159,0.0,0.069,0.221,0.11,3.426,72,819,1
4,0.00,2.08,0.00,0.0,3.12,0.00,1.04,0.00,0.00,0.00,...,0.000,0.000,0.0,0.263,0.000,0.00,1.428,4,20,1


In [24]:
X = spam_fused.drop("Class", axis=1)
y = spam_fused["Class"]

In [110]:
#Splitting dataset into trainig set and testing set, 20-80. About 1000 training and 3600 testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [111]:
# creating the classifiers

gnb_clf = GaussianNB()
dt_clf = DecisionTreeClassifier(criterion = "entropy")
lr_clf = LogisticRegression(max_iter=2000 )

In [112]:
# voting classifier

vote = VotingClassifier(estimators = [('DT', dt_clf),('GNB',gnb_clf),('LR',lr_clf)], voting='hard')

In [113]:
vote.fit(X_train,y_train)

VotingClassifier(estimators=[('DT',
                              DecisionTreeClassifier(criterion='entropy')),
                             ('GNB', GaussianNB()),
                             ('LR', LogisticRegression(max_iter=2000))])

In [114]:
vote_pred = vote.predict(X_test)

### Classification Accuracy Vote:

In [115]:
#Evaluating the model, looking at classification accuracy

print("Classification Accuracy: ", accuracy_score(y_test,vote_pred))

Classification Accuracy:  0.9305103148751357


### Per-Class Classification Accuracy Vote:

In [116]:
#using classification_report to find per-class classification accuracy
print("Per-class Classification Accuracy: \n\n", classification_report(y_test,vote_pred))

Per-class Classification Accuracy: 

               precision    recall  f1-score   support

           0       0.96      0.93      0.94       555
           1       0.89      0.94      0.91       366

    accuracy                           0.93       921
   macro avg       0.93      0.93      0.93       921
weighted avg       0.93      0.93      0.93       921



### Confusion Matrix Vote:

In [117]:
#using confusion matrix on the test instances

matrix_vote = confusion_matrix(y_test,vote_pred)
print(matrix_vote)

[[514  41]
 [ 23 343]]


## Part 2: Adaboost with Decision Tree as Base Learner

In [118]:
ada_clf = AdaBoostClassifier(n_estimators = 200,base_estimator=dt_clf)

In [119]:
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy'),
                   n_estimators=200)

In [120]:
ada_pred = ada_clf.predict(X_test)

### Classification Accuracy AdaBoost:

In [121]:
#Evaluating the model, looking at classification accuracy

print("Classification Accuracy: ", accuracy_score(y_test,ada_pred))

Classification Accuracy:  0.9370249728555917


### Per-Class Classification Accuracy AdaBoost:

In [122]:
#using classification_report to find per-class classification accuracy
print("Per-class Classification Accuracy: \n\n", classification_report(y_test,ada_pred))

Per-class Classification Accuracy: 

               precision    recall  f1-score   support

           0       0.95      0.95      0.95       555
           1       0.92      0.92      0.92       366

    accuracy                           0.94       921
   macro avg       0.93      0.93      0.93       921
weighted avg       0.94      0.94      0.94       921



### Confusion Matrix AdaBoost:

In [123]:
#using confusion matrix on the test instances

matrix_ada = confusion_matrix(y_test,ada_pred)
print(matrix_ada)

[[527  28]
 [ 30 336]]


## Part 3: Random Forest:

In [55]:
rf_clf = RandomForestClassifier()

In [56]:
rf_clf.fit(X_train,y_train)

RandomForestClassifier()

In [57]:
rf_pred = rf_clf.predict(X_test)

### Classification Accuracy Random Forest:

In [58]:
#Evaluating the model, looking at classification accuracy

print("Classification Accuracy: ", accuracy_score(y_test,rf_pred))

Classification Accuracy:  0.9296386851399077


### Per-Class Classification Accuracy Random Forest:

In [64]:
#using classification_report to find per-class classification accuracy
print("Per-class Classification Accuracy: \n\n", classification_report(y_test,rf_pred))

Per-class Classification Accuracy: 

               precision    recall  f1-score   support

           0       0.94      0.94      0.94      2232
           1       0.91      0.91      0.91      1449

    accuracy                           0.93      3681
   macro avg       0.93      0.93      0.93      3681
weighted avg       0.93      0.93      0.93      3681



### Confusion Matrix Random Forest:

In [59]:
#using confusion matrix on the test instances

matrix_rf = confusion_matrix(y_test,rf_pred)
print(matrix_rf)

[[2100  132]
 [ 127 1322]]
